In [ ]:
import os
from os import listdir
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

import cv2
from matplotlib.image import imread




import glob
import PIL



In [ ]:
!pip install kaggle
!pip install albumentations opencv-python matplotlib

In [ ]:
import os
import cv2
import numpy as np
import albumentations as A
import matplotlib.pyplot as plt0
from tqdm import tqdm
import random

In [ ]:
dataset_folder = '/content/dataset_folder'


In [ ]:
import os
import json

kaggle_dir = os.path.expanduser("~/.kaggle")
os.makedirs(kaggle_dir, exist_ok=True)

# Move kaggle.json to the correct location
with open("/content/kaggle.json", "r") as f:
    kaggle_credentials = json.load(f)

with open(os.path.join(kaggle_dir, "kaggle.json"), "w") as f:
    json.dump(kaggle_credentials, f)

os.chmod(os.path.join(kaggle_dir, "kaggle.json"), 0o600)  # Secure file permissions

In [ ]:
import os
import pandas as pd

# Define dataset folder
dataset_folder = "/content/dataset_folder"
os.makedirs(dataset_folder, exist_ok=True)  # Ensure the folder exists

# Download and extract dataset
!kaggle datasets download -d awsaf49/cbis-ddsm-breast-cancer-image-dataset -p "{dataset_folder}"
!unzip -q "{dataset_folder}/cbis-ddsm-breast-cancer-image-dataset.zip" -d "{dataset_folder}"


In [ ]:
dataset_folder = '/content/dataset_folder'


In [ ]:
dicom_data = pd.read_csv(os.path.join(dataset_folder, 'csv', 'dicom_info.csv'))

mass_case_train_df = pd.read_csv(os.path.join(dataset_folder, 'csv', 'mass_case_description_train_set.csv'))
mass_case_test_df = pd.read_csv(os.path.join(dataset_folder, 'csv', 'mass_case_description_test_set.csv'))





In [ ]:
mass_case_df = pd.concat([mass_case_train_df, mass_case_test_df], ignore_index=True)

In [ ]:
image_dir = os.path.join(dataset_folder, 'jpeg')  # Extract images here


In [ ]:
dicom_data.head()

In [ ]:
dicom_data.info()

In [ ]:
unique_series_descriptions = dicom_data['SeriesDescription'].unique()
print(unique_series_descriptions)


In [ ]:
full_image_df = dicom_data[dicom_data['SeriesDescription'] == 'full mammogram images']
full_image_df['image_path'] = full_image_df['image_path'].apply(lambda x: x.replace('CBIS-DDSM/jpeg', image_dir))

In [ ]:
mass_case_df['SeriesInstanceUID'] = mass_case_df['image file path'].apply(lambda path: path.split('/')[2])

In [ ]:
map_image_path_series_id = dict(zip(full_image_df['SeriesInstanceUID'].values,full_image_df['image_path'].values))

In [ ]:
mass_case_df['image_path'] = mass_case_df['SeriesInstanceUID'].apply(lambda seri: map_image_path_series_id[seri])
mass_case_df.head(1)

In [ ]:
mass_case_df['pathology'].unique()


In [ ]:
mass_case_df['pathology'].value_counts()

In [ ]:
import os
import cv2
import pandas as pd
import albumentations as A
from tqdm import tqdm

ALLOWED_EXTENSIONS = {'jpg', 'jpeg', 'png', 'bmp', 'tiff'}

mammography_augmentations = {


    "GaussNoise": A.GaussNoise(var_limit=(5.0, 15.0), p=1.0),
    "RandomBrightnessContrast": A.RandomBrightnessContrast(brightness_limit=0.05, contrast_limit=0.05, p=1.0),
    "RandomGamma": A.RandomGamma(gamma_limit=(98, 102), p=1.0),
    "Equalize": A.Equalize(p=1.0),
    "GaussianBlur": A.GaussianBlur(blur_limit=1, p=1.0),

}

def apply_augmentations(image, augmentations):
    augmented_images = {}
    for aug_name, aug in augmentations.items():
        augmented = aug(image=image)
        augmented_images[aug_name] = augmented['image']
    return augmented_images

def balance_classes_mass(dataset_folder, images_folder, output_folder, dataframe):
    class_counts = dataframe["pathology"].value_counts()
    min_class_count = class_counts.min()
    max_class_count = class_counts.max()

    os.makedirs(output_folder, exist_ok=True)
    final_data = []

    for pathology in dataframe["pathology"].unique():
        df_class = dataframe[dataframe["pathology"] == pathology]
        real_images = df_class.sample(n=min_class_count, replace=False).reset_index(drop=True)
        real_images_copy = real_images.copy()

        num_augmented_needed = max_class_count - min_class_count
        augmented_data = []
        aug_index = 0

        print(f"{pathology}: Keeping {min_class_count} real images, generating {num_augmented_needed} augmented.")

        while len(augmented_data) < num_augmented_needed:
            row = real_images.iloc[aug_index % len(real_images)]
            image_path = row['image_path']

            if not os.path.exists(image_path):
                print(f"Image not found: {image_path}")
                aug_index += 1
                continue

            image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            if image is None:
                print(f"Failed to load image: {image_path}")
                aug_index += 1
                continue

            image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)

            for aug_name, aug in mammography_augmentations.items():
                if len(augmented_data) >= num_augmented_needed:
                    break

                augmented = aug(image=image)['image']
                new_image_name = f"aug_{aug_name}_{aug_index}_{os.path.basename(image_path)}"
                new_image_path = os.path.join(output_folder, new_image_name)
                cv2.imwrite(new_image_path, augmented)

                new_row = row.copy()
                new_row['image_path'] = new_image_path
                augmented_data.append(new_row)

            aug_index += 1

        final_data.extend(real_images_copy.to_dict('records'))
        final_data.extend(augmented_data)

    final_df = pd.DataFrame(final_data)
    final_df.to_csv(os.path.join(dataset_folder, "augmented_balanced_mass_dataset.csv"), index=False)
    print("Balanced dataset saved with real = min count and total = max count per class.")

In [ ]:
balance_classes_mass(dataset_folder='/content/dataset_folder',
                images_folder="os.path.join(dataset_folder, 'jpeg')",
                output_folder="augmented_masss_images",
                dataframe=mass_case_df)


In [ ]:
aug_df = pd.read_csv("/content/dataset_folder/augmented_balanced_mass_dataset.csv")



In [ ]:
aug_df['pathology'].value_counts()

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd

aug_df = pd.read_csv("/content/dataset_folder/augmented_balanced_mass_dataset.csv")
train_df, test_df = train_test_split(aug_df, test_size=0.2, shuffle=True, random_state=42)


In [ ]:
train_df.head()

In [ ]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
import pickle


# Recreate and fit the encoders
label_encoder_abnormality = LabelEncoder()
label_encoder_pathology = LabelEncoder()

label_encoder_abnormality.fit(train_df["abnormality type"])
label_encoder_pathology.fit(train_df["pathology"])

# Save the encoders to disk
with open("label_encoder_abnormality.pkl", "wb") as f:
    pickle.dump(label_encoder_abnormality, f)
with open("label_encoder_pathology.pkl", "wb") as f:
    pickle.dump(label_encoder_pathology, f)

print("Encoders saved successfully as 'label_encoder_abnormality.pkl' and 'label_encoder_pathology.pkl'")

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, Model
from tensorflow.keras.applications import DenseNet121

df = train_df
df = df[df["abnormality type"] == "mass"]

label_encoder_pathology = LabelEncoder()
df["pathology_encoded"] = label_encoder_pathology.fit_transform(df["pathology"])
num_classes_pathology = len(label_encoder_pathology.classes_)
df["pathology_encoded"] = df["pathology_encoded"].apply(lambda x: tf.keras.utils.to_categorical(x, num_classes=num_classes_pathology))

class SingleOutputDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, df, batch_size, img_size=(256, 256), shuffle=True):
        self.df = df.reset_index(drop=True)
        self.batch_size = batch_size
        self.img_size = img_size
        self.shuffle = shuffle
        self.indices = np.arange(len(self.df))
        self.on_epoch_end()
    def __len__(self):
        return len(self.df) // self.batch_size
    def __getitem__(self, index):
        idx = self.indices[index*self.batch_size:(index+1)*self.batch_size]
        batch = self.df.iloc[idx]
        X = np.array([img_to_array(load_img(p, target_size=self.img_size)) for p in batch["image_path"]]) / 255.0
        y = np.stack(batch["pathology_encoded"].values)
        return X, y
    def on_epoch_end(self):
        if self.shuffle:
            np.random.shuffle(self.indices)

base = DenseNet121(input_shape=(256, 256, 3), include_top=False, weights="imagenet")
base.trainable = False

x = base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(512, activation='relu')(x)
x = layers.BatchNormalization()(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(num_classes_pathology, activation='softmax', name='pathology')(x)

model = Model(inputs=base.input, outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

train_gen = SingleOutputDataGenerator(df, batch_size=32)

model.fit(train_gen, epochs=10)
model.save("densenet121.h5")


In [ ]:
# Assume test_df is defined and contains columns: 'image_path', 'abnormality type', 'pathology'
test_df = test_df[test_df["abnormality type"] == "mass"]

# Use the same label encoder as training
test_df["pathology_encoded"] = label_encoder_pathology.transform(test_df["pathology"])
test_df["pathology_encoded"] = test_df["pathology_encoded"].apply(
    lambda x: tf.keras.utils.to_categorical(x, num_classes=num_classes_pathology)
)

# Create test data generator
test_gen = SingleOutputDataGenerator(test_df, batch_size=32, shuffle=False)

# Evaluate model
loss, accuracy = model.evaluate(test_gen)
print(f"Test Accuracy: {accuracy * 100:.2f}%")


In [ ]:
from google.colab import files
files.download("densenet121.h5")


In [ ]:
from google.colab import files
files.download("/content/label_encoder_pathology.pkl")